# CALMS21 Investigation NEAR Results Analysis

This notebook analyzes and compares results from:
- Baseline NEAR experiments (if available)
- Neurosym-lib NEAR experiments (from results pickle files)

## Step 0: Data collection.

Compute results for neurosym-lib experiments
```bash
uv run python notebooks/calms21_reproduction/benchmark_calms21.py
```


In [1]:
%load_ext autoreload
%autoreload 2
%cd /home/asehgal/neurosym-lib

import json
import pickle
from pathlib import Path

import numpy as np
import pandas as pd

from neurosym.examples.near.metrics import compute_metrics

/home/asehgal/neurosym-lib


/home/asehgal/neurosym-lib/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
/home/asehgal/neurosym-lib/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Baseline Results

In [2]:
# Load best NEAR results
near_results_pth = Path("outputs/calms21_results/near_outputs/")

baseline_results = {}
for result_file in near_results_pth.rglob("test_results.json"):
    print(f"Loading Baseline results from {result_file}")
    with open(result_file, "r") as f:
        expt_report = json.load(f)
        experiment_name = result_file.parent.name
        y_true = np.array(expt_report["true_vals"])
        y_scores = np.array(expt_report["predicted_vals"])
        expt_report["report"] = compute_metrics(y_scores, y_true)
        baseline_results[experiment_name] = expt_report

## Load Neurosym-lib Results

In [3]:
file_path = "outputs/calms21_results/reproduction.pkl"
neurosym_results = {}
if Path(file_path).exists():
    with open(file_path, "rb") as f:
        results = pickle.load(f)
        file_name = Path(file_path).stem
        for i, result in enumerate(results):
            key = f"neurosym_{file_name}_{i:03d}"
            neurosym_results[key] = result
    print(f"  Loaded {len(results)} programs from {file_name}")
else:
    print(f"Warning: {file_path} not found")

## Reported Program


In [ ]:
reported_path = "outputs/calms21_results/reported_program.pkl"
if Path(reported_path).exists():
    with open(reported_path, "rb") as f:
        reported_program_result = pickle.load(f)
    print(f"Loaded reported program results from {reported_path}")
else:
    print(f"Warning: {reported_path} not found")
    reported_program_result = None

## Create Comparison Table

In [ ]:
# Combine all results
all_results = {}
all_results.update(baseline_results)
all_results.update(neurosym_results)

if reported_program_result is not None:
    all_results["reported_program"] = reported_program_result

print(f"Total experiments: {len(all_results)}")

In [ ]:
# Create comparison dataframe
table_data = []

for name, result in all_results.items():
    # Extract metrics
    report = result.get("report", {})
    
    # Handle different report structures
    if isinstance(report, dict):
        if "report" in report:
            macro_avg = report["report"].get("macro avg", {})
            hamming_acc = report.get("hamming_accuracy", 0.0)
        elif "macro avg" in report:
            macro_avg = report["macro avg"]
            hamming_acc = report.get("hamming_accuracy", 0.0)
        else:
            # Need to compute metrics
            if "pred_vals" in result and "true_vals" in result:
                pred_vals = np.array(result["pred_vals"])
                true_vals = np.array(result["true_vals"])
                metrics = compute_metrics(pred_vals, true_vals)
                macro_avg = metrics["report"]["macro avg"]
                hamming_acc = metrics["hamming_accuracy"]
            else:
                continue
    else:
        continue
    
    row = {
        "experiment": name,
        "precision": macro_avg.get("precision", 0.0),
        "recall": macro_avg.get("recall", 0.0),
        "f1_score": macro_avg.get("f1-score", 0.0),
        "support": macro_avg.get("support", 0),
        "hamming_accuracy": hamming_acc,
        "time": result.get("time", 0.0),
    }
    table_data.append(row)

df = pd.DataFrame(table_data)

# Sort by hamming_accuracy descending
df = df.sort_values("hamming_accuracy", ascending=False)

print("\n" + "=" * 80)
print("RESULTS COMPARISON")
print("=" * 80)
print(df.to_string(index=False))
print("=" * 80)

## Summary Statistics

In [ ]:
print("\nSUMMARY STATISTICS")
print("=" * 80)
print(f"Total experiments: {len(df)}")
print(f"\nBest F1-score: {df['f1_score'].max():.6f}")
print(f"  Experiment: {df.loc[df['f1_score'].idxmax(), 'experiment']}")
print(f"\nBest Hamming accuracy: {df['hamming_accuracy'].max():.6f}")
print(f"  Experiment: {df.loc[df['hamming_accuracy'].idxmax(), 'experiment']}")
print(f"\nAverage F1-score: {df['f1_score'].mean():.6f}")
print(f"Average Hamming accuracy: {df['hamming_accuracy'].mean():.6f}")
print(f"Average training time: {df['time'].mean():.2f} seconds")

# Baseline vs Neurosym comparison
baseline_df = df[df['experiment'].str.contains('baseline|calms21_astar|calms21_iddfs|enumeration', case=False, na=False)]
neurosym_df = df[df['experiment'].str.contains('neurosym|reproduction', case=False, na=False)]

if not baseline_df.empty:
    print(f"\nBaseline experiments: {len(baseline_df)}")
    print(f"  Avg F1: {baseline_df['f1_score'].mean():.6f}")
    print(f"  Avg Hamming Acc: {baseline_df['hamming_accuracy'].mean():.6f}")
    
if not neurosym_df.empty:
    print(f"\nNeurosym-lib experiments: {len(neurosym_df)}")
    print(f"  Avg F1: {neurosym_df['f1_score'].mean():.6f}")
    print(f"  Avg Hamming Acc: {neurosym_df['hamming_accuracy'].mean():.6f}")

# Reported program
reported_df = df[df['experiment'] == 'reported_program']
if not reported_df.empty:
    print("\nReported Program:")
    print(f"  F1: {reported_df['f1_score'].values[0]:.6f}")
    print(f"  Hamming Acc: {reported_df['hamming_accuracy'].values[0]:.6f}")
    print(f"  Time: {reported_df['time'].values[0]:.2f}s")

print("=" * 80)

## Save Results

In [ ]:
# Save CSV
csv_path = "outputs/calms21_results/comparison.csv"
Path(csv_path).parent.mkdir(parents=True, exist_ok=True)
df.to_csv(csv_path, index=False)
print(f"Saved CSV to: {csv_path}")

# Save Markdown
md_path = "outputs/calms21_results/comparison.md"
with open(md_path, "w") as f:
    f.write("# CALMS21 Investigation NEAR Results Comparison\n\n")
    f.write(df.to_markdown(index=False))
    f.write("\n")
print(f"Saved Markdown to: {md_path}")

## Display Top Programs

In [ ]:
# Show top 10 programs by F1 score
print("\nTop 10 Programs by F1-score:")
print("=" * 80)
top_10 = df.head(10)
print(top_10.to_string(index=False))

# Show program strings for top results if available
print("\nProgram Details:")
print("=" * 80)
for idx, row in top_10.iterrows():
    exp_name = row['experiment']
    if exp_name in all_results:
        result = all_results[exp_name]
        program_str = result.get('program_str', str(result.get('program', 'N/A')))
        print(f"\n{exp_name}:")
        print(f"  Program: {program_str}")
        print(f"  F1: {row['f1_score']:.6f}, Hamming Acc: {row['hamming_accuracy']:.6f}, Time: {row['time']:.2f}s")